In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from yomi.historical_record import fetch_historical_elo, compute_elo as yomi_compute_elo
from bacon import load_historical_record, compute_elo as bacon_compute_elo

In [11]:
compute_elo = yomi_compute_elo
matches = fetch_historical_elo()
matches.head()


,set_number,event,match_date,elo_1_before,set_win_1,player_1,player_2,set_win_2,elo_2_before
0,1,Summer Smash,2014-07-06,1500.0,1,ntillerman,Felix,0,1500.0
1,2,Summer Smash,2014-07-06,1500.0,1,Mallorean_Thug,Nathan,0,1500.0
2,3,Summer Smash,2014-07-06,1500.0,1,Bob199,Djister,0,1500.0
3,4,Summer Smash,2014-07-06,1500.0,1,Xom,rybalko,0,1500.0
4,5,Summer Smash,2014-07-06,1500.0,1,Redless,LK4O4,0,1500.0


In [5]:
compute_elo = bacon_compute_elo
matches = load_historical_record()
matches.head()

ERROR:root:Unable to understand ../bacon-replays/Archive-1553479911441/a5b5f81f-da9d-4edd-b164-7fb576558274_replay.bcr
Traceback (most recent call last):
  File "/code/bacon.py", line 57, in load_replay_results
    if replay.fighter_0.name <= replay.fighter_1.name
  File "/usr/local/lib/python3.7/site-packages/bacon_replay_analyzer/__init__.py", line 135, in fighter_0
    setup = next(self.parsed_tuples)
  File "/usr/local/lib/python3.7/site-packages/bacon_replay_analyzer/__init__.py", line 222, in parsed_tuples
    for index, tuple in self.raw_tuples:
  File "/usr/local/lib/python3.7/site-packages/bacon_replay_analyzer/__init__.py", line 198, in raw_tuples
    yield ("SETUP", self.raw_tuple("GAME_SETUP"))
  File "/usr/local/lib/python3.7/site-packages/bacon_replay_analyzer/__init__.py", line 182, in raw_tuple
    section = self.parsed[timestamp]
  File "/usr/local/lib/python3.7/configparser.py", line 958, in __getitem__
    raise KeyError(key)
KeyError: 'GAME_SETUP'
/usr/local/lib/pyt

,character_1,character_2,match_date,player_1,player_1_losses,player_1_ties,player_1_wins,player_2,player_2_losses,player_2_ties,player_2_wins,record_1,record_2,replay,submitter,timestamp,win,player_1_games_played,player_2_games_played
0,Magdelina,Seven,2018-01-10 20:54:34,Jugan,0.0,0.0,0.0,Aliphant,0.0,0.0,0.0,NaN,NaN,d4da4ce8-c2e4-4101-bf15-ec214091a308_replay.bcr,Aliphant,NaN,False,0.0,0.0
1,Kimbhe,Magdelina,2018-01-10 21:19:34,Aliphant,0.0,0.0,0.0,Jugan,0.0,0.0,0.0,NaN,NaN,9507085b-dff9-4c10-aa9c-b4c027764aa3_replay.bcr,Aliphant,NaN,True,0.0,0.0
2,Karin,Shekhtur,2018-01-10 23:31:14,AliceXVI,0.0,0.0,0.0,Aliphant,0.0,0.0,0.0,NaN,NaN,48135afb-1e8c-4262-a240-caa7f3410088_replay.bcr,Aliphant,NaN,False,0.0,0.0
3,Karin,Khadath,2018-01-11 06:56:24,Mickamax,0.0,0.0,0.0,Aliphant,0.0,0.0,0.0,NaN,NaN,8c0de64f-5a03-49df-b6f9-8c97eadb0060_replay.bcr,Aliphant,NaN,False,0.0,0.0
4,Karin,Magdelina,2018-01-11 08:22:54,Winged,0.0,0.0,0.0,Aliphant,0.0,0.0,0.0,NaN,NaN,01f54762-c42b-4ed2-a4ac-4d437fbc7a30_replay.bcr,Aliphant,NaN,False,0.0,0.0


In [16]:
results = []

def score_elo(elo):
    elo["points_diff"] = elo.elo_before_1 - elo.elo_before_2
    elo["win_chance"] = 1 / (1 + (-elo.points_diff / 1135.77).rpow(10))
    squared_error = (
        (elo.win_chance - elo.win).pow(2)
        + ((1 - elo.win_chance) - (1-elo.win)).pow(2)
    ).sum()

    return squared_error

current_elo = compute_elo(matches, low_k=25, high_k=125, k_cutoff=95)
best_result = (score_elo(current_elo), 25, 125, 95)
print("Previous Best Result", best_result)

for low_k in range(45, 65, 1):
    for high_k in range(70, 90, 1):
        for k_cutoff in range(180, 200, 1):
            elo = compute_elo(matches, low_k=low_k, high_k=high_k, k_cutoff=k_cutoff)
            score = score_elo(elo)
            result = (score, low_k, high_k, k_cutoff)
            if best_result is None or result < best_result:
                best_result = result
                print("Found new best result", result)
            results.append(result)
            if low_k == high_k:
                break

Previous Best Result (6178.629107236963, 25, 125, 95)
Found new best result (6133.204438380506, 45, 70, 180)
Found new best result (6132.839212157836, 45, 70, 181)
Found new best result (6132.801614163282, 45, 70, 183)
Found new best result (6132.431072648429, 45, 70, 184)
Found new best result (6132.010655603642, 45, 70, 186)
Found new best result (6131.771569091951, 45, 71, 186)
Found new best result (6131.5631931242, 45, 72, 186)
Found new best result (6131.384595120871, 45, 73, 186)
Found new best result (6131.234885235121, 45, 74, 186)
Found new best result (6131.113213791801, 45, 75, 186)
Found new best result (6131.018768912049, 45, 76, 186)
Found new best result (6130.950774307928, 45, 77, 186)
Found new best result (6130.908487233128, 45, 78, 186)
Found new best result (6130.891196576971, 45, 79, 186)
Found new best result (6130.869910827727, 46, 77, 186)
Found new best result (6130.826624187373, 46, 78, 186)
Found new best result (6130.808332330487, 46, 79, 186)
Found new bes

KeyboardInterrupt: 

In [15]:
min(results)

(6131.3975321319485, 55, 80, 190)

In [17]:
result

(6132.204947027818, 59, 86, 193)